<a href="https://colab.research.google.com/github/tng00/Machine_Learning_MAI_2023/blob/main/homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашняя работа по регуляризации и оптимизации

Ниже приводится корпус данных с двумя метками: 1 и -1. К данным применяется линейная модель классификации:

$f(x, \theta) = x_1 \theta_1 + x_2 \theta_2 + \theta_3.$

Предлагается подобрать параметры $\theta$ минимизируя следующую функцию ошибки:

$\mathcal{L}(\theta) = 0.1 \|\theta\|^2 + \frac{1}{N}\sum\limits_{i=1}^N \max(0, 1 - y_i f(x_i, \theta)).$

Для оптимизации предлагается использовать метод градиентного спуска с 1000 шагами размера $0.1$ из начальной точки $(1, 1, 0)$.

In [1]:
import numpy as np
import yaml

In [2]:
X = np.array([
    [0, 1],
    [1, 1],
    [1, 0],
    [-0.5, 0.5],
    [0, -0.5]
])

y = np.array([1, 1, 1, -1, -1])

theta0 = np.array([1.0, 1.0, 0.0])

lr = 0.1

def f(X, theta):
    theta = np.asarray(theta)
    return (X * theta[:2]).sum(axis=-1) + theta[2]

def loss(X, y, theta):
    theta = np.asarray(theta)
    norm = (theta ** 2).sum()
    deltas = y * f(X, theta)
    return 0.1 * norm + np.mean(np.maximum(0, 1 - deltas))

print("Prediction:", f(X, theta0))
print("Loss:", loss(X, y, theta0))

Prediction: [ 1.   2.   1.   0.  -0.5]
Loss: 0.5


# Градиент функции потерь
$$\mathfrak{L}(\theta) = 0.1 ||\theta||^2 + \frac{1}{N} \sum^N_{i=1} \max(0, 1-y_i f(x_i, \theta))  $$

$$f(x, \theta) = x_1 \theta_1 + x_2 \theta_2 + \theta_3 $$

Будем считать градиент относительно $θ$. 

Далее, верхним индексом будем обозначать столбцы, нижним - строки.

Пусть $ϕ(\theta_i, j) = max(0, 1 - y_j \cdot φ_j(\theta)), \ \ φ_j(\theta) = f(x_j, \theta), \ \ i \in \{1..3\}, \ \ j \in \{1..N\}$

Для начала, посчитаем производные $φ_j(\theta)$: \\
$$\frac{∂φ_j}{∂θ_1} = x^1_j$$
$$\frac{∂φ_j}{∂θ_2} = x^2_j$$
$$\frac{∂φ_j}{∂θ_3} = 1$$
$$\nabla φ_j = \begin{pmatrix} x^1_j \\ x^2_j \\ 1 \end{pmatrix}$$

По правилу дифференцирования сложной функции:
$$\frac{∂ϕ}{∂θ} = \frac{∂\phi}{∂φ}\frac{∂φ}{∂θ}$$

Отсюда: \\
$$\xi_i := \frac{\partial ϕ}{∂\theta_i} = 
\begin{cases}
  0, & y_j \cdot φ_j(θ) > 1 \\
  - y_j x^i_j, & y_j \cdot φ_j(θ) < 1
\end{cases}$$

(полагая $x^3_j$ = 1) \\

Вычислим производные нормы $||\theta||^2$:
$$||\theta||^2 = \theta_1^2 + \theta_2^2 + \theta_3^2 $$

$$\frac{\partial ||\theta||^2}{∂θ_i} = 2\theta_i$$
$$\nabla ||\theta||^2 = \begin{pmatrix} 2θ_1 \\ 2θ_2 \\ 2θ_3 \end{pmatrix}$$

Итого:
$$\frac{\partial \mathfrak{L}}{∂\theta_i} = 0.2 \theta_i + \frac{1}{N} \sum_{j=1}^N \xi^i_j$$

In [3]:
theta = theta0
def gradient(X, y, theta):
    der_theta = np.asarray(theta)
    deltas = y * f(X, der_theta)
    ext_der = (deltas < 1).astype(int) * (-y)
    der_mean = np.zeros_like(der_theta)
    der_mean[:2] = np.mean(ext_der[:, np.newaxis] * X, axis=0)
    der_mean[2] = np.mean(ext_der)
    return 0.1 * 2 * der_theta + der_mean

for i in range(1000):
    grad = gradient(X, y, theta)
    theta -= lr * grad

In [5]:
print("Optimized theta:", theta0)
print("Prediction:", f(X, theta))
print("Loss:", loss(X, y, theta))

with open("submission.yaml", "w") as fp:
    yaml.safe_dump({"tasks": [{"task1": {"answer": theta.tolist()}}]}, fp)

Optimized theta: [ 1.25218494  1.         -0.24781506]
Prediction: [ 0.75218494  2.00436988  1.00436988 -0.37390753 -0.74781506]
Loss: 0.48815643632236133
